In [15]:
scope.dis()
target.dis()

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!


In [4]:
exit()

In [3]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

In [3]:
load(scope, "masked-decode-inside", None)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12355 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12355 bytes


In [13]:
scope.default_setup()
scope.adc.samples=24400
scope.clock.adc_src = "clkgen_x1"
scope.adc.decimate = 2

In [12]:
load(scope, "masked-decode-inside-test", None)
time.sleep(0.1)
target.flush()
scope.arm()
w = [0, 0, 0, 0, 0, 0] + [0 for i in range(250)]
target.simpleserial_write('p', get_bytes(w))
ret = scope.capture()
if ret:
    print("ERROR")
traces_ = scope.get_last_trace()
db = target.simpleserial_read('r', 16)
print(scope.adc.trig_count)


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12443 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12443 bytes
40577


In [6]:
posture(target,'a', bytearray([0]*4), 4)

bytearray(b'\x00\x00\x00\x00')

In [16]:

# function
def setup():
    # runs once in the start -> prints returned
    # load(scope, "../c-files/???", env)
    load(scope, "masked-decode-inside-test", env)
    time.sleep(0.1)
    
    # print(date_str())
    pass

current_key = None
current_rand = None
def execute(index, op, t):
    global current_key, current_rand

    FI_HD = [0 for i in range(127)]
    FI_PUF = [0 for i in range(127)]
    if op == "0":
        posture(target,'a', bytearray([0]*4), 4)
    elif op =="1":
        posture(target,'a', bytearray([127]*4), 4)
    else:
        raise "ERROR"
    p_temp = p0.sample(byte=False)
    k_temp = k0
    
        
    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k_temp), FI_HD)
    posture(target,'k', get_bytes(mem), 16)
    p_temp = xor(FI_PUF, p_temp)
    current_key = k_temp
    current_rand = rand
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    if scope.adc.trig_count > scope.adc.samples * env.segments:
        print(scope.adc.trig_count)
    if current_key != xor(current_rand,get_bits(fdb)[:64]):
        print(s.encode(current_key))
        print(current_key)
        print(get_bits(fdb))
        print("ahHAHAHA")
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass


s = Scheme() # scheme execution helper


p0 = [random.randrange(2) for _ in range(127)]
k0 = [random.randrange(2) for _ in range(64)]
unstable = None
p0 = PUF(p0, unstable)

env = ENV()
# operation settings
prefix_name = "masked"
env.root = "D:/programs/traces/MASKED"
env.execute = execute
env.ending = ending
env.jupyter = "test_Test" # current notebook name
env.operations = operations = ["0","1"] # operations list
inv = lambda x: operations.index(x)
env.subfolder = "test" # subfolder name
env.TRACES_NUM = 100 # number of traces
scope.adc.samples = 24400 # number of samples per execution
env.segments = 3 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = ""
env.graph = {"type":"time", "xlines": [950, 60900]}


scope.clock.adc_src = "clkgen_x1"
scope.adc.decimate = 1

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()
print(start_time)


file_names = [f"file{i}" for i in range(5)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)


i = 0

i += 1
env.name = prefix_name+f"_{i}"
env.title = "Dividers"
env.desc = ""
create_tasker(env, file_names, [inv("0"), inv("1")], f"group{i}")



env.name = prefix_name
#remove_files(env, file_names)
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


#print(s.encode(current_key))
#print(current_key)
#print(get_bits(fdb))
#next experiment: 

AttributeError: 'NoneType' object has no attribute 'controlWrite'

In [ ]:
execute mission
# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()
for i in range(1,6):
    status = traces("file_name", 0)
split_file(["file_name"], [0, 1], "result")
remove_files(["file_name"])
generate_file("start_time", "operations")
print("done")